# 02 GEE Map Greenland_a
This is a simple exerciese of mapping Greenland with Google Earth Engine.

In [9]:
# import pandas as pd
import ee
# if in colab:
# !pip install geemap
import geemap
import geemap.colormaps as cm
# import geemap.eefolium as geemap
# ee.Initialize()

## Greenland: Arctic DEM

In [10]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map # comment this line if in colab

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [11]:


# greenlandmask = ee.Image('OSU/GIMP/2000_ICE_OCEAN_MASK') \
#                    .select('ice_mask').eq(1); #'ice_mask', 'ocean_mask'
arcticDEM = ee.Image('UMN/PGC/ArcticDEM/V3/2m_mosaic')
mask = arcticDEM.gt(0)
arcticDEMgreenland = arcticDEM.updateMask(mask)

# palette = cm.get_palette('terrain', n_class=30)

visPara = {'min': 0,  'max': 5971.24, 'palette': 'white'}
# visPara = {'min': 0,  'max': 2500.0, 'palette': palette}
# Map.addLayer(arcticDEMgreenland, visPara, 'Arctic DEM')
Map.addLayer(arcticDEMgreenland, visPara, 'Arctic DEM extent')
Map.setCenter(-41.0, 74.0, 3)
#add colorbar
# colors = visPara['palette']
# vmin = visPara['min']
# vmax = visPara['max']

# Map.add_colorbar(visPara, label="Elevation (m a.s.l)", orientation="vertical", layer_name="Arctic DEM")
# Map.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, caption="m a.s.l")


## Greenland: ArcticDEM contourlines

In [12]:


palette = cm.get_palette('terrain', n_class=14)
visPara = {'min': 0,  'max': 3500, 'palette': palette}

lines = ee.List.sequence(0, 3500, 250)

def func_contourline(line):
  mycontour = arcticDEMgreenland \
    .convolve(ee.Kernel.gaussian(5, 3)) \
    .subtract(ee.Image.constant(line)).zeroCrossing() \
    .multiply(ee.Image.constant(line)).toFloat()

  return mycontour.mask(mycontour)

contourlines = lines.map(func_contourline)

contourlines = ee.ImageCollection(contourlines).mosaic()
Map.addLayer(contourlines, visPara, 'contours')
Map.add_colorbar(visPara, label="Elevation (m a.s.l)", discrete=True, orientation="horizontal", layer_name="countours")